# Итоговое задание №2. Прогнозирование конечных свойств новых материалов (композиционных материалов)
На входе имеются данные о начальных свойствах компонентов композиционных материалов (количество связующего, наполнителя, температурный режим отверждения и т.д.). На выходе необходимо спрогнозировать ряд конечных свойств получаемых композиционных материалов. 

Предсказание (из семинара 22): 
* модуль упругости при растяжении
* прочности при растяжении
* cоотношение матрица-наполнитель

Предсказание (из итогового вебинара 26.11.2022):
* модуль упругости при растяжении
* прочности при растяжении

# Разделение данных на обучающую, тестовую и валидационную выборки

In [1]:
import numpy as np
import pandas as pd
import sklearn as sl

In [2]:
df0_norm = pd.read_csv('../data/interim/df0_norm.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла
df90_norm = pd.read_csv('../data/interim/df90_norm.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла

In [4]:
#df0_norm.info()
df90_norm.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 20 to 1022
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Соотношение матрица-наполнитель       503 non-null    float64
 1   Плотность, кг/м3                      503 non-null    float64
 2   модуль упругости, ГПа                 503 non-null    float64
 3   Количество отвердителя, м.%           503 non-null    float64
 4   Содержание эпоксидных групп,%_2       503 non-null    float64
 5   Температура вспышки, С_2              503 non-null    float64
 6   Поверхностная плотность, г/м2         503 non-null    float64
 7   Модуль упругости при растяжении, ГПа  503 non-null    float64
 8   Прочность при растяжении, МПа         503 non-null    float64
 9   Потребление смолы, г/м2               503 non-null    float64
 10  Шаг нашивки                           503 non-null    float64
 11  Плотность нашивки

In [9]:
#работа дальнейшая с датафреймом для угла нашивки 0 градусов
y_df_0elastic =  pd.DataFrame(df0_norm['Модуль упругости при растяжении, ГПа'],
                            columns=['Модуль упругости при растяжении, ГПа'])
y_df_0strength = pd.DataFrame(df0_norm['Прочность при растяжении, МПа'],
                            columns=['Прочность при растяжении, МПа'])
y_df_0matrix = pd.DataFrame(df0_norm['Соотношение матрица-наполнитель'],
                            columns=['Соотношение матрица-наполнитель'])

                            #работа дальнейшая с датафреймом для угла нашивки 90 градусов
y_df_90elastic =  pd.DataFrame(df90_norm['Модуль упругости при растяжении, ГПа'],
                            columns=['Модуль упругости при растяжении, ГПа'])
y_df_90strength = pd.DataFrame(df90_norm['Прочность при растяжении, МПа'],
                            columns=['Прочность при растяжении, МПа'])
y_df_90matrix = pd.DataFrame(df90_norm['Соотношение матрица-наполнитель'],
                            columns=['Соотношение матрица-наполнитель'])

In [11]:
x_df_0elastic = df0_norm.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1)
x_df_0strength = df0_norm.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1)
x_df_0matrix = df0_norm.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1)

x_df_90elastic = df90_norm.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1)
x_df_90strength = df90_norm.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1)
x_df_90matrix = df90_norm.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1)


Разделение и сохранение датасетов на Train ,test, valid

In [12]:
from sklearn.model_selection import train_test_split

def Divide_2_train_test_val(x_df, y_df, nm_str):
    X_train_val, X_test, y_train_val, y_test = train_test_split(x_df.values,
                            y_df.values, test_size=.3, random_state=17)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                            y_train_val, test_size=.3, random_state=17)

    print(f'Количество объектов в тренировочных данных {nm_str}: {X_train.shape[0]}')
    print(f'Количество объектов в тестовых данных {nm_str}: {X_test.shape[0]}')
    print(f'Количество объектов в валидационных данных {nm_str}: {X_val.shape[0]}') 

    df_save = pd.DataFrame(X_train, columns=x_df.columns)
    df_save.to_csv('../data/interim/X_train_' + nm_str +'.csv', sep=';')

    df_save = pd.DataFrame(X_test, columns=x_df.columns)
    df_save.to_csv('../data/interim/X_test_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(X_val, columns=x_df.columns)
    df_save.to_csv('../data/interim/X_val_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(y_train, columns=y_df.columns)
    df_save.to_csv('../data/interim/y_train_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(y_test, columns=y_df.columns)
    df_save.to_csv('../data/interim/y_test_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(y_val, columns=y_df.columns)
    df_save.to_csv('../data/interim/y_val_'+nm_str+'.csv', sep=';')

In [13]:
Divide_2_train_test_val(x_df_0elastic, y_df_0elastic, '0_elastic')
Divide_2_train_test_val(x_df_0strength, y_df_0strength, '0_strength')
Divide_2_train_test_val(x_df_0matrix, y_df_0matrix, '0_matrix')

Divide_2_train_test_val(x_df_90elastic, y_df_90elastic, '90_elastic')
Divide_2_train_test_val(x_df_90strength, y_df_90strength, '90_strength')
Divide_2_train_test_val(x_df_90matrix, y_df_90matrix, '90_matrix')

Количество объектов в тренировочных данных 0_elastic: 254
Количество объектов в тестовых данных 0_elastic: 156
Количество объектов в валидационных данных 0_elastic: 110
Количество объектов в тренировочных данных 0_strength: 254
Количество объектов в тестовых данных 0_strength: 156
Количество объектов в валидационных данных 0_strength: 110
Количество объектов в тренировочных данных 0_matrix: 254
Количество объектов в тестовых данных 0_matrix: 156
Количество объектов в валидационных данных 0_matrix: 110
Количество объектов в тренировочных данных 90_elastic: 246
Количество объектов в тестовых данных 90_elastic: 151
Количество объектов в валидационных данных 90_elastic: 106
Количество объектов в тренировочных данных 90_strength: 246
Количество объектов в тестовых данных 90_strength: 151
Количество объектов в валидационных данных 90_strength: 106
Количество объектов в тренировочных данных 90_matrix: 246
Количество объектов в тестовых данных 90_matrix: 151
Количество объектов в валидационных

y_column_names = ['Модуль упругости при растяжении, ГПа','Прочность при растяжении, МПа','Соотношение матрица-наполнитель']
x_column_names = ['Плотность, кг/м3', 'модуль упругости, ГПа','Количество отвердителя, м.%','Содержание эпоксидных групп,%_2','Температура вспышки, С_2','Поверхностная плотность, г/м2','Потребление смолы, г/м2','Угол нашивки, град','Шаг нашивки','Плотность нашивки']

dfX = df.drop(columns=y_column_names)
dfY = df.drop(columns=x_column_names)